This notebook analyses the total generation (in MWh), total generation revenue (in AUD) and the volume-weighted average price for various fuel types in the National Electricity Market (NEM).

The input data includes:
1. Spot prices for all regions of the NEM, with 5-minute resolution, covering the period from FY2020 to FY2024.
    -- ##Production\Report\Electricity\Generation\Actual\Region\Metered\Generation Stack FuelType ScheduleType (5min)
2. Dispatched electricity by fuel type across all NEM regions, with 5-minute resolution, for the same FY2020 to FY2024 period.
    -- ##Production\Report\Electricity\Price\Actual\NEM\Dispatch (5min)\Dispatch Price all regions (5min)

These analyses aim to provide insights into generation performance and pricing dynamics by fuel type in the NEM.

In [1]:
import pandas as pd
from os import path
import numpy as np
import glob

In [2]:
input_path = r'C:\Users\wwang2\Documents\GitHub\NEM-generation\data'
output_path = r'C:\Users\wwang2\Documents\GitHub\NEM-generation\results'

In [3]:
# Define the ToD function to add time-related columns
def ToD(df):
    df['DateTime'] = pd.to_datetime(df['DateTime'], format='%d/%m/%Y %H:%M:%S')
    df['Month'] = df['DateTime'].dt.month
    df['CY'] = df['DateTime'].dt.year
    df['Quarter'] = df['DateTime'].dt.quarter
    df['FY'] = df['DateTime'].map(lambda d: d.year + 1 if d.month > 6 else d.year)
    df['Hour'] = df['DateTime'].dt.hour

In [4]:
# Load the dispatch price data
price_df = pd.read_csv(path.join(input_path, 'Spot Price_2019_2024.csv'))

In [5]:
# Create a mapping of state to the corresponding dispatch price column
state_price_col = {
    'NSW': 'NSW1 DISPATCH_PRICE',
    'QLD': 'QLD1 DISPATCH_PRICE',
    'VIC': 'VIC1 DISPATCH_PRICE',
    'SA':  'SA1 DISPATCH_PRICE',
    'TAS': 'TAS1 DISPATCH_PRICE'
}

In [6]:
# Process each generation file
state_list = ['NSW', 'QLD', 'VIC', 'SA', 'TAS']
generation_types = ['Schedule', 'Semi_Schedule']

In [7]:
# Create a list to store aggregated results
aggregated_results = []

for state in state_list:
    for gen_type in generation_types:
        # Generate the file pattern for each state's generation files (Schedule and Semi_Schedule)
        file_pattern = f'{state}1_{gen_type}_2019_2024_5_minutes.csv'
        
        # Load the generation data for the current state and generation type
        try:
            gen_df = pd.read_csv(path.join(input_path, file_pattern))
            # print("found file: ", file_pattern)
        except FileNotFoundError:
            raise FileNotFoundError(f'File not found: {file_pattern}')
        
        # Add time-related columns using the ToD function
        ToD(gen_df)

        gen_df['DateTime'] = pd.to_datetime(gen_df['DateTime'], format='%d/%m/%Y %H:%M')
        price_df['DateTime'] = pd.to_datetime(price_df['DateTime'], format='%d/%m/%Y %H:%M')
        
        # Merge the generation data with the dispatch price data
        merged_df = pd.merge(gen_df, price_df[['DateTime', state_price_col[state]]], on='DateTime', how='inner')

        # Calculate MWh dispatched by dividing MW by 12, while keeping DISPATCH_MW
        for col in merged_df.columns:
            if 'DISPATCH_MW' in col:  # Identify generation columns
                merged_df[col.replace('DISPATCH_MW', 'DISPATCH_MWh')] = merged_df[col] / 12
        
        # Calculate revenue: MWh dispatched * dispatch price for the state
        dispatch_price_col = state_price_col[state]
        for col in merged_df.columns:
            if 'DISPATCH_MWh' in col:  # Identify MWh columns
                fuel_type = col.replace(' DISPATCH_MWh', '')  # Extract fuel type
                merged_df[f'{fuel_type} REVENUE'] = merged_df[col] * merged_df[dispatch_price_col]

        # Aggregate the data at the level of financial year, quarter, month, hour
        for col in merged_df.columns:
            if 'DISPATCH_MWh' in col:  # Identify MWh columns
                fuel_type = col.replace(' DISPATCH_MWh', '')  # Extract fuel type
                # Group by time dimensions and calculate total MWh, total revenue, and VWAP
                agg_df = merged_df.groupby(['FY']).agg(
                    Total_MWh_Dispatched=(col, 'sum'),
                    Total_Revenue=(f'{fuel_type} REVENUE', 'sum')
                ).reset_index()
                
                # Calculate VWAP (Total Revenue / Total MWh Dispatched)
                agg_df['VWAP'] = agg_df['Total_Revenue'] / agg_df['Total_MWh_Dispatched']
                agg_df['State'] = state
                agg_df['Fuel_Type'] = fuel_type
                
                # Store the aggregated data in the results list
                aggregated_results.append(agg_df)

In [8]:
# Concatenate all aggregated results into a single DataFrame
final_agg_df = pd.concat(aggregated_results)

# Calculate the sum of 'Total_MWh_Dispatched' and 'Total_Revenue' for each financial year
sum_mwh_fy = final_agg_df.groupby('FY')['Total_MWh_Dispatched'].transform('sum')
sum_revenue_fy = final_agg_df.groupby('FY')['Total_Revenue'].transform('sum')

# Calculate the new columns
final_agg_df['% of total MWh dispatched'] = final_agg_df['Total_MWh_Dispatched'] / sum_mwh_fy * 100
final_agg_df['% of total revenue'] = final_agg_df['Total_Revenue'] / sum_revenue_fy * 100

# Display the updated dataframe
print(final_agg_df.head())

     FY  Total_MWh_Dispatched  Total_Revenue        VWAP State Fuel_Type  \
0  2020              0.000000   0.000000e+00         NaN   NSW   Battery   
1  2021              0.000000   0.000000e+00         NaN   NSW   Battery   
2  2022          13780.540833   5.091473e+06  369.468286   NSW   Battery   
3  2023          22553.717500   6.361942e+06  282.079513   NSW   Battery   
4  2024         102064.612500   2.842369e+07  278.487248   NSW   Battery   

   % of total MWh dispatched  % of total revenue  
0                   0.000000            0.000000  
1                   0.000000            0.000000  
2                   0.007433            0.019770  
3                   0.012205            0.024232  
4                   0.055171            0.159216  


In [9]:
# Save the aggregated result to a new CSV file
final_agg_df.to_csv(path.join(output_path, 'NEM_generation.csv'), index=False)

In [10]:
# Create a list to store aggregated results
aggregated_results_ToD = []

for state in state_list:
    for gen_type in generation_types:
        # Generate the file pattern for each state's generation files (Schedule and Semi_Schedule)
        file_pattern = f'{state}1_{gen_type}_2019_2024_5_minutes.csv'
        
        # Load the generation data for the current state and generation type
        try:
            gen_df = pd.read_csv(path.join(input_path, file_pattern))
            # print("found file: ", file_pattern)
        except FileNotFoundError:
            raise FileNotFoundError(f'File not found: {file_pattern}')
        
        # Add time-related columns using the ToD function
        ToD(gen_df)

        gen_df['DateTime'] = pd.to_datetime(gen_df['DateTime'], format='%d/%m/%Y %H:%M')
        price_df['DateTime'] = pd.to_datetime(price_df['DateTime'], format='%d/%m/%Y %H:%M')
        
        # Merge the generation data with the dispatch price data
        merged_df = pd.merge(gen_df, price_df[['DateTime', state_price_col[state]]], on='DateTime', how='inner')

        # Calculate MWh dispatched by dividing MW by 12, while keeping DISPATCH_MW
        for col in merged_df.columns:
            if 'DISPATCH_MW' in col:  # Identify generation columns
                merged_df[col.replace('DISPATCH_MW', 'DISPATCH_MWh')] = merged_df[col] / 12
        
        # Calculate revenue: MWh dispatched * dispatch price for the state
        dispatch_price_col = state_price_col[state]
        for col in merged_df.columns:
            if 'DISPATCH_MWh' in col:  # Identify MWh columns
                fuel_type = col.replace(' DISPATCH_MWh', '')  # Extract fuel type
                merged_df[f'{fuel_type} REVENUE'] = merged_df[col] * merged_df[dispatch_price_col]

        # Aggregate the data at the level of financial year, quarter, month, hour
        for col in merged_df.columns:
            if 'DISPATCH_MWh' in col:  # Identify MWh columns
                fuel_type = col.replace(' DISPATCH_MWh', '')  # Extract fuel type
                # Group by time dimensions and calculate total MWh, total revenue, and VWAP
                agg_df = merged_df.groupby(['FY', 'Hour']).agg(
                    Total_MWh_Dispatched=(col, 'sum'),
                    Total_Revenue=(f'{fuel_type} REVENUE', 'sum')
                ).reset_index()
                
                # Calculate VWAP (Total Revenue / Total MWh Dispatched)
                agg_df['VWAP'] = agg_df['Total_Revenue'] / agg_df['Total_MWh_Dispatched']
                agg_df['State'] = state
                agg_df['Fuel_Type'] = fuel_type
                
                # Store the aggregated data in the results list
                aggregated_results_ToD.append(agg_df)

In [11]:
# Concatenate all aggregated results into a single DataFrame
final_agg_df_ToD = pd.concat(aggregated_results_ToD)

# Calculate the sum of 'Total_MWh_Dispatched' and 'Total_Revenue' for each financial year and hour
sum_mwh_fy_ToD = final_agg_df_ToD.groupby(['FY', 'Hour'])['Total_MWh_Dispatched'].transform('sum')
sum_revenue_fy_ToD = final_agg_df_ToD.groupby(['FY', 'Hour'])['Total_Revenue'].transform('sum')

# Calculate the new columns
final_agg_df_ToD['% of total MWh dispatched'] = final_agg_df_ToD['Total_MWh_Dispatched'] / sum_mwh_fy_ToD * 100
final_agg_df_ToD['% of total revenue'] = final_agg_df_ToD['Total_Revenue'] / sum_revenue_fy_ToD * 100

# Display the updated dataframe
print(final_agg_df_ToD.head())

     FY  Hour  Total_MWh_Dispatched  Total_Revenue  VWAP State Fuel_Type  \
0  2020     0                   0.0            0.0   NaN   NSW   Battery   
1  2020     1                   0.0            0.0   NaN   NSW   Battery   
2  2020     2                   0.0            0.0   NaN   NSW   Battery   
3  2020     3                   0.0            0.0   NaN   NSW   Battery   
4  2020     4                   0.0            0.0   NaN   NSW   Battery   

   % of total MWh dispatched  % of total revenue  
0                        0.0                 0.0  
1                        0.0                 0.0  
2                        0.0                 0.0  
3                        0.0                 0.0  
4                        0.0                 0.0  


In [12]:
# Save the aggregated result to a new CSV file
final_agg_df_ToD.to_csv(path.join(output_path, 'NEM_generation_ToD.csv'), index=False)